La librería python-jobspy agrega resultados de LinkedIn, Indeed, Glassdoor y ZipRecruiter sin que tengas que configurar Selenium.

In [16]:
#Librerías necesarias
import pandas as pd
from jobspy import scrape_jobs
import time
import random

In [17]:
# 1. Configuración de la fragmentación
locations = ["Málaga, Spain", "Granada, Spain"]
search_terms = ["Data Engineer", "Software Developer"] # Lista de puestos
df_presencial = []

print("Iniciando búsqueda fragmentada y anidada...")

for loc in locations:
    for term in search_terms:
        print(f"--- Buscando: '{term}' en {loc} ---")
        
        try:
            jobs = scrape_jobs(
                site_name=["linkedin"],
                search_term=term,
                location=loc,
                results_wanted=40,
                is_remote=False,             
                linkedin_fetch_description=True 
            )
            
            df_res = pd.DataFrame(jobs)
            if not df_res.empty:
                # Añadimos metadatos para saber de qué búsqueda vino cada fila
                df_res['search_location'] = loc
                df_res['search_query'] = term
                df_presencial.append(df_res)
                print(f"   Sucesos: {len(df_res)} ofertas encontradas.")
            else:
                print(f"   Sin resultados para '{term}' en {loc}.")

        except Exception as e:
            print(f"Error en búsqueda '{term}' en {loc}: {e}")

        # Pausa entre combinaciones de puesto/ciudad para no saturar a LinkedIn
        wait_time = random.uniform(12, 25)
        print(f"Esperando {wait_time:.2f} segundos para la siguiente combinación...\n")
        time.sleep(wait_time)

# 2. Consolidación final
if df_presencial:
    df_final = pd.concat(df_presencial, ignore_index=True)
    
    # Limpieza: Eliminar duplicados por URL
    # Muy importante: una misma oferta puede aparecer para ambos términos de búsqueda
    total_antes = len(df_final)
    df_final = df_final.drop_duplicates(subset=['job_url'])
    total_despues = len(df_final)
    
    print(f"Proceso finalizado.")
    print(f"Total bruto: {total_antes} | Únicos: {total_despues}")
else:
    print("No se pudo recolectar ninguna oferta.")

Iniciando búsqueda fragmentada y anidada...
--- Buscando: 'Data Engineer' en Málaga, Spain ---
   Sucesos: 40 ofertas encontradas.
Esperando 13.41 segundos para la siguiente combinación...

--- Buscando: 'Software Developer' en Málaga, Spain ---
   Sucesos: 40 ofertas encontradas.
Esperando 24.28 segundos para la siguiente combinación...

--- Buscando: 'Data Engineer' en Granada, Spain ---
   Sucesos: 40 ofertas encontradas.
Esperando 20.45 segundos para la siguiente combinación...

--- Buscando: 'Software Developer' en Granada, Spain ---
   Sucesos: 40 ofertas encontradas.
Esperando 23.43 segundos para la siguiente combinación...

Proceso finalizado.
Total bruto: 160 | Únicos: 142


In [18]:
import pandas as pd
import random
import time
from jobspy import scrape_jobs

# 1. Configuración para la búsqueda en remoto
print("--- Iniciando búsqueda de ofertas en REMOTO ---")

search_terms = ["Data Engineer", "Software Developer"]
jobs_remote_list = []

for term in search_terms:
    print(f"--- Buscando Remoto: '{term}' ---")
    try:
        jobs_remote = scrape_jobs(
            site_name=["linkedin"],
            search_term=term,
            location="Spain",           
            results_wanted=40,           
            is_remote=True,              
            linkedin_fetch_description=True 
        )
        
        df_temp = pd.DataFrame(jobs_remote)
        
        if not df_temp.empty:
            df_temp['search_location'] = 'Remote (Spain)'
            df_temp['search_query'] = term
            jobs_remote_list.append(df_temp)
            print(f"   Sucesos: {len(df_temp)} ofertas en remoto encontradas.")
        else:
            print(f"   No se encontraron ofertas en remoto para '{term}'.")

    except Exception as e:
        print(f"   Error en la búsqueda remota de '{term}': {e}")

    # Pausa de seguridad entre términos
    wait_time = random.uniform(15, 25)
    print(f"Esperando {wait_time:.2f} segundos para el siguiente perfil...\n")
    time.sleep(wait_time)

# 2. Consolidación y Limpieza
if jobs_remote_list:
    df_remote_total = pd.concat(jobs_remote_list, ignore_index=True)
    df_final = pd.concat([df_final, df_remote_total], ignore_index=True)

--- Iniciando búsqueda de ofertas en REMOTO ---
--- Buscando Remoto: 'Data Engineer' ---
   Sucesos: 40 ofertas en remoto encontradas.
Esperando 20.63 segundos para el siguiente perfil...

--- Buscando Remoto: 'Software Developer' ---
   Sucesos: 40 ofertas en remoto encontradas.
Esperando 21.82 segundos para el siguiente perfil...



In [19]:
# Limpieza de duplicados por URL
antes = len(df_final)
df_final = df_final.drop_duplicates(subset=['job_url'])
despues = len(df_final)

print(f"--- Proceso de consolidación finalizado ---")
print(f"Total bruto (Presencial + Remoto): {antes}")
print(f"Total tras limpiar duplicados: {despues}")

--- Proceso de consolidación finalizado ---
Total bruto (Presencial + Remoto): 222
Total tras limpiar duplicados: 221


In [20]:
import os
from datetime import datetime

# 1. Obtener la fecha para la carpeta (dd-mm-yyyy)
fecha_hoy = datetime.now().strftime("%d-%m-%Y")

# 2. Obtener el timestamp para el nombre del archivo (HH-MM)
timestamp_archivo = datetime.now().strftime("%H-%M")

# 3. Definir la ruta completa: scraps/dd-mm-yyyy/
# Usamos os.path.join para que funcione bien en cualquier sistema operativo
ruta_carpeta = os.path.join('scraps', fecha_hoy)

# 4. Crear la carpeta (y las carpetas padre si no existen)
if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

# 5. Definir el nombre del archivo y guardar
nombre_archivo = f"ofertas_it_{timestamp_archivo}.csv"
ruta_final = os.path.join(ruta_carpeta, nombre_archivo)

if not df_final.empty:
    df_final.to_csv(ruta_final, index=False, encoding='utf-8')
    print(f"--- Datos guardados en: {ruta_final} ---")
else:
    print("El DataFrame está vacío, no se guardó el archivo.")

--- Datos guardados en: scraps/15-02-2026/ofertas_it_19-25.csv ---
